In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import src.tokens as tokenization_utils

torch.__version__, transformers.__version__, torch.version.cuda

('2.1.2+cu121', '4.39.0.dev0', '12.1')

In [3]:
from src.models import ModelandTokenizer

MODEL_PATH = "state-spaces/mamba-2.8b" # state-spaces/mamba-2.8b | state-spaces/mamba-2.8b-slimpj


mt = ModelandTokenizer(
    model_path=MODEL_PATH, 
    torch_dtype=torch.float32
)

/home/local_arnab/miniconda3/envs/relations/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [120]:
subject = "The Space Needle"
prompt_template = "{} is located in the city of"


prompt = tokenization_utils.maybe_prefix_eos(mt, prompt_template.format(subject))

mt.reset_forward()
functional.predict_next_token(
    mt = mt, 
    prompt = prompt,
)

x.shape=torch.Size([1, 11, 5120])
x_dbl.shape=torch.Size([1, 11, 192])
delta.shape=torch.Size([1, 11, 160]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16])
x.shape=torch.Size([1, 11, 5120])
x_dbl.shape=torch.Size([1, 11, 192])
delta.shape=torch.Size([1, 11, 160]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16])
x.shape=torch.Size([1, 11, 5120])
x_dbl.shape=torch.Size([1, 11, 192])
delta.shape=torch.Size([1, 11, 160]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16])
x.shape=torch.Size([1, 11, 5120])
x_dbl.shape=torch.Size([1, 11, 192])
delta.shape=torch.Size([1, 11, 160]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16])
x.shape=torch.Size([1, 11, 5120])
x_dbl.shape=torch.Size([1, 11, 192])
delta.shape=torch.Size([1, 11, 160]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16])
x.shape=torch.Size([1, 11, 5120])
x_dbl.shape=torch.Size([1, 11, 192])
delta.shape=torch.Size([1, 11, 160]), B.shape=torch.Size([1

[[PredictedToken(token=' Seattle', prob=0.9869956374168396),
  PredictedToken(token='\n', prob=0.0030492732767015696),
  PredictedToken(token='  ', prob=0.0009398188558407128),
  PredictedToken(token=' downtown', prob=0.0008882731199264526),
  PredictedToken(token=' Tac', prob=0.0006469952641054988)]]

In [5]:
from src.hooking.mamba import selective_scan_with_mask

tokenized = mt.tokenizer(prompt, return_tensors="pt", padding=True, return_offsets_mapping=True).to(mt.device)
offsets = tokenized.pop("offset_mapping")

subj_first, subj_last = functional.find_token_range(
    string = prompt,
    substring=subject,
    offset_mapping=offsets[0],
)

subj_last -= 1
last_idx = tokenized.input_ids.shape[1] - 1

In [6]:
import types

mamba_block_format = "layers.{}.mixer"


def mask_layers(model, layer_indices, mask_config):
    masked_scan = lambda self, u, delta, A, B, C, D: selective_scan_with_mask(
        self=self, u=u, delta=delta, A=A, B=B, C=C, D=D, mask=mask_config
    )
    for layer_idx in layer_indices:
        block = baukit.get_module(model, mamba_block_format.format(layer_idx))
        block.selective_scan = types.MethodType(masked_scan, block)

# -----------------------------------------
mt.reset_forward()
# -----------------------------------------

mask_everything = {
    idx: torch.arange(0, idx + 1).tolist() for idx in range(last_idx + 1)
}

mask_layers(
    model=mt.model,
    # layer_indices = [20],
    layer_indices=torch.arange(0, mt.n_layer).tolist(),
    # mask_config = {last_idx: []}  # no mask
    # mask_config = {last_idx: [subj_last]}   # last token cant see subject_last token
    # mask_config = {last_idx: torch.arange(subj_first, subj_last + 1).tolist()} # last token cant see the entire subject
    # mask_config={
    #     target_idx: [subj_last]
    #     for target_idx in range(subj_last + 1, last_idx + 1)
    # },  # the entire query cant see subject_last
    mask_config={
        target_idx: torch.arange(subj_first, subj_last + 1).tolist()
        for target_idx in range(subj_last + 1, last_idx + 1)
    },  # the entire query cant see the entire subject
    # mask_config = mask_everything
)

In [7]:
# mt.reset_forward()

# ! just ablating the diagonal ssm isn't enough.
# TODO: figure out how to ablate the shift-SSM or the conv as well
# also, the "attention" visualization is wrong, because it doesn't take the Conv into account
# technically, ssm doesn't pay attention on a particular token, it pays attention to the entire receptive field

functional.predict_next_token(
    mt = mt, 
    prompt = prompt,
)

deltaA.shape=torch.Size([1, 11, 5120, 16]) | deltaB_u.shape=torch.Size([1, 11, 5120, 16])
y.norm()=tensor(5.3791, device='cuda:0')
||y_0|| = 3.772832155227661 | IS IT ZERO: False | max = 2.942206621170044 | min = -1.0835286378860474
||y_1|| = 0.5428782105445862 | IS IT ZERO: False | max = 0.1441761553287506 | min = -0.19203220307826996
||y_2|| = 1.3580889701843262 | IS IT ZERO: False | max = 0.15453104674816132 | min = -0.23775498569011688
||y_3|| = 2.6891846656799316 | IS IT ZERO: False | max = 0.20786795020103455 | min = -0.2433052361011505
||y_4|| = 1.6687171459197998 | IS IT ZERO: False | max = 0.10465343296527863 | min = -0.14150381088256836
||y_5|| = 0.30827596783638 | IS IT ZERO: False | max = 0.06259013712406158 | min = -0.10572979599237442
||y_6|| = 1.1205065250396729 | IS IT ZERO: False | max = 0.12731100618839264 | min = -0.2169119417667389
||y_7|| = 0.33747056126594543 | IS IT ZERO: False | max = 0.03542409837245941 | min = -0.06401839107275009
||y_8|| = 0.2079717069864273 

[[PredictedToken(token=' Seattle', prob=0.973701536655426),
  PredictedToken(token=' Eve', prob=0.002749845152720809),
  PredictedToken(token=' Belle', prob=0.002449068007990718),
  PredictedToken(token=' Tac', prob=0.0021701513323932886),
  PredictedToken(token=' Vancouver', prob=0.0015045679174363613)]]

In [8]:
from src.utils import experiment_utils
experiment_utils.set_seed(123456)

u = torch.randn(1, 4, 5120)
delta = torch.randn(1, 4, 5120)
A = torch.randn(5120, 16)
B = torch.randn(1, 4, 16)
C = torch.randn(1, 4, 16)
D = torch.randn(5120)

output = selective_scan_with_mask(
    self=None,
    u=u,
    delta=delta,
    A=A,
    B=B,
    C=C,
    D=D,
    # mask = {0:[]}
    mask={0: [0], 1: [0, 1], 2: [0, 1, 2], 3: [0, 1, 2, 3]},
)

output.shape

deltaA.shape=torch.Size([1, 4, 5120, 16]) | deltaB_u.shape=torch.Size([1, 4, 5120, 16])
y.norm()=tensor(1.1922)
||y_0|| = 0.0 | IS IT ZERO: True | max = 0.0 | min = 0.0
||y_1|| = 0.00020085324649699032 | IS IT ZERO: True | max = 4.7594308853149414e-05 | min = -0.00010585784912109375
||y_2|| = 0.007619727402925491 | IS IT ZERO: False | max = 0.0027017593383789062 | min = -0.0063304901123046875
||y_3|| = 1.1921932697296143 | IS IT ZERO: False | max = 1.1787971258163452 | min = -0.17270827293395996
-------------------------------------


torch.Size([1, 4, 5120])

In [9]:
delta_A = torch.randn(1, 4, 5120, 16)
src_idx = 2
target_idx = 2

delta_A_src_to_target = torch.prod(delta_A[:, src_idx + 1 : target_idx + 1], dim=1)
delta_A_src_to_target.shape

torch.Size([1, 5120, 16])

In [10]:
delta_B_src = torch.randn(1, 5120, 16)

In [11]:
delta_AB_src = delta_A_src_to_target * delta_B_src

torch.allclose(delta_B_src, delta_AB_src)

True

In [12]:
mt.model

Mamba(
  (embedding): Embedding(50280, 2560)
  (layers): ModuleList(
    (0-63): 64 x ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
        (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
        (x_proj): Linear(in_features=5120, out_features=192, bias=False)
        (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
        (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
      )
      (norm): RMSNorm()
    )
  )
  (norm_f): RMSNorm()
  (lm_head): Linear(in_features=2560, out_features=50280, bias=False)
)

In [13]:
block = baukit.get_module(
    mt.model,
    "layers.10.mixer",
)

block.conv1d.weight.shape, block.conv1d.bias.shape

(torch.Size([5120, 1, 4]), torch.Size([5120]))

In [14]:
l = 16
b = 1
x = torch.randn(b, 5120, l).to(mt.device)

In [15]:
y_conv = block.conv1d(x)[:, :, :l]
y_conv.shape

torch.Size([1, 5120, 16])

In [16]:
def convolve_one_filter(
    x: torch.Tensor, 
    filter_weights: torch.Tensor, 
    filter_bias: torch.Tensor, 
    prefix_padding: int = block.conv1d.padding[0],
):
    x = torch.nn.functional.pad(x, (prefix_padding, 0))
    y = torch.zeros_like(x)
    print(f"{x.shape=}, {filter_weights.shape=}, {filter_bias.shape=}, {y.shape=}, {prefix_padding=}")
    for idx in range(filter_weights.shape[0], x.shape[1]):
        y[:, idx] = torch.sum(x[:, idx - filter_weights.shape[0] : idx] * filter_weights, dim=1) + filter_bias
    return y[:, : x.shape[1] - prefix_padding]

filter_idx = 20
inp_channel = 20
f_weights = block.conv1d.weight[filter_idx]
f_bias = block.conv1d.bias[filter_idx]
y_conv_manual = convolve_one_filter(x[:, inp_channel], f_weights, f_bias)

x[:, 0].shape, y_conv_manual.shape


x.shape=torch.Size([1, 19]), filter_weights.shape=torch.Size([1, 4]), filter_bias.shape=torch.Size([]), y.shape=torch.Size([1, 19]), prefix_padding=3


(torch.Size([1, 16]), torch.Size([1, 16]))

In [17]:
y_conv_manual

tensor([[ 0.0000, -0.0528, -0.0528, -0.0528, -0.1643, -0.2462, -0.0537,  0.0573,
         -0.0358, -0.1461, -0.0175, -0.2345, -0.1679,  0.0419, -0.0855, -0.1168]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [18]:
y_conv[:, inp_channel]

tensor([[-0.0885, -0.1570, -0.1471, -0.0667, -0.0279, -0.0558, -0.0611, -0.1131,
         -0.1626, -0.0959, -0.0696, -0.0812, -0.0671, -0.0429, -0.0671, -0.0890]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [89]:
def conv_manual(x, weight, bias, padding):
    pad = (padding[0], 0) 
    x_padded = torch.nn.functional.pad(x, pad, 'constant', 0) 
    
    batch_size, channels, width = x.shape
    out_channels, in_channels_per_group, kernel_size = weight.shape
    output_width = width - kernel_size + 1 + sum(pad)
    y = torch.zeros((batch_size, out_channels, output_width))

    for batch in range(batch_size):
        # for channel in range(channels):
        #     for i in range(output_width):
        #         current_slice = x_padded[batch, channel, i:i+kernel_size]
        #         y[batch, channel, i] = (current_slice * weight[channel]).sum() + bias[channel]
        for i in range(output_width):
            current_slice = x_padded[batch, :, i:i+kernel_size]
            y[batch, :, i] = (current_slice * weight.squeeze(dim=1)).sum(dim=1) + bias

    return y


conv1d = block.conv1d

In [90]:
l = 16
b = 1
x = torch.randn(b, 5120, l).to(mt.device)

In [91]:
y_conv = conv1d(x)[:, :, :l]

In [92]:
y_conv_manual = conv_manual(x, conv1d.weight, conv1d.bias, padding=conv1d.padding)

In [93]:
y_conv

tensor([[[-0.0115,  0.0384, -0.1258,  ..., -0.0823,  0.2133,  0.0356],
         [ 0.0286, -0.0265, -0.0096,  ..., -0.0423, -0.1671, -0.0367],
         [ 0.0199, -0.0121,  0.1716,  ...,  0.0635,  0.1294, -0.0111],
         ...,
         [-0.0211, -0.0619,  0.0695,  ..., -0.0637, -0.1161, -0.1475],
         [-0.1147,  0.0076, -0.0324,  ..., -0.1727, -0.2052, -0.0666],
         [-0.0030, -0.0416, -0.0124,  ..., -0.0209, -0.0121, -0.1551]]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [94]:
y_conv_manual

tensor([[[-0.0115,  0.0384, -0.1258,  ..., -0.0823,  0.2133,  0.0356],
         [ 0.0286, -0.0265, -0.0096,  ..., -0.0423, -0.1671, -0.0367],
         [ 0.0199, -0.0121,  0.1716,  ...,  0.0635,  0.1294, -0.0111],
         ...,
         [-0.0211, -0.0619,  0.0695,  ..., -0.0637, -0.1161, -0.1475],
         [-0.1147,  0.0076, -0.0324,  ..., -0.1727, -0.2052, -0.0666],
         [-0.0030, -0.0416, -0.0124,  ..., -0.0209, -0.0121, -0.1551]]],
       grad_fn=<CopySlices>)

In [101]:
y_conv.shape, y_conv_manual.shape

(torch.Size([1, 5120, 16]), torch.Size([1, 5120, 16]))

In [102]:
x_padded = torch.nn.functional.pad(x, (conv1d.padding[0], 0), 'constant', 0)

In [103]:
channel = 2
batch_idx = 0
x_slice = x_padded[batch_idx, channel, 1:5]
(x_slice * conv1d.weight[channel]).sum() + conv1d.bias[channel]

tensor(-0.0121, device='cuda:0', grad_fn=<AddBackward0>)

In [104]:
batch_idx = 0
x_slice = x_padded[batch_idx, :, 1:5]
# (x_slice * conv1d.weight).sum(dim=1) + conv1d.bias

In [108]:
# x_slice.shape, conv1d.weight.shape

(x_slice * conv1d.weight.squeeze(1)).shape

torch.Size([5120, 4])

In [106]:
(x_slice * conv1d.weight.squeeze(1)).sum(dim=1) + conv1d.bias

tensor([ 0.0384, -0.0265, -0.0121,  ..., -0.0619,  0.0076, -0.0416],
       device='cuda:0', grad_fn=<AddBackward0>)

In [107]:
x_padded.shape, x_slice.shape

(torch.Size([1, 5120, 19]), torch.Size([5120, 4]))

In [119]:
block.x_proj.weight.shape, block.dt_proj.weight.shape

(torch.Size([192, 5120]), torch.Size([5120, 160]))

In [117]:
dummy_x = torch.randn(1, 16, 5120).to(mt.device)
block.x_proj(dummy_x).shape

torch.Size([1, 16, 192])

In [115]:
mt.model.args.dt_rank + 16 * 2

192